In [38]:
from fuzzywuzzy import fuzz
import sqlite3
import pandas as pd 

def get_data_from_db(query, database='database.db'):
    conn = sqlite3.connect(database)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

def identify_matching_value_sale_label(label, list_, threshold=90):
    list_similarity = [
        (txt, fuzz.ratio(txt.lower(), sub_label.lower())) 
        for txt in list_ 
        for sub_label in label.split(' ')
    ]
    
    if max(list_similarity, key=lambda x: x[1])[1] > threshold:
        return max(list_similarity, key=lambda x: x[1])[0]
    else:
        return ''

def match_sale_to_product_sheet(sale_label):
    """
    Input : label of the sale 
    Output : product sheet id linked to the sale 
    """
    category_identified, brand_identified = "", ""

    # 0. verification and pre-processing on the input 

    # 1. identify the category of the product (phone, monitor, ...)
    df_category = get_data_from_db("SELECT * FROM category", database='database.db')
    list_category = list(df_category.category_label.values)
    category_identified = identify_matching_value_sale_label(sale_label, list_category, threshold=90)
    
    if category_identified == "":
        return False
    
    id_category = df_category[df_category.category_label == category_identified].category_id.values[0]

    # 2. identify the brand of the product (Apple, LG, ...)
    # the category identified should be removed from the sale_label
    query_brand = """
        SELECT brand.* FROM brand  
        JOIN product  
            ON brand.brand_id = product.brand_id
        WHERE category_id = {}
    """.format(id_category) 
    
    df_brand = get_data_from_db(query_brand, database='database.db')
    list_brand = list(df_brand.brand_label.values)

    if len(list_brand): 
        brand_identified = identify_matching_value_sale_label(sale_label, list_brand, threshold=90)

    # 3. identify the feature of the product (ultrawide, version 2, ...)

    # 4. find the product sheet matching those caracteristics 
    query_product = "SELECT product_label FROM product WHERE category_id = {}".format(id_category)
    if brand_identified != '':
        id_brand = df_brand[df_brand.brand_label == brand_identified].brand_id.values[0]
        query_product += " AND brand_id = {}".format(id_brand)

    df_product = get_data_from_db(query_product, database='database.db')

    print('Input : ', sale_label)
    print('. Category identified : ', category_identified)
    print('. Brand identified : ', brand_identified)

    if df_product.shape[0] == 1:
        return df_product.product_label.values[0]
    else: 
        return False
    

product_identified = match_sale_to_product_sheet('Apple Iphone pro 14')
print('Output : ', product_identified)


Input :  Apple Iphone pro 14
. Category identified :  phone
. Brand identified :  apple
Output :  iPhone


In [40]:
product_identified = match_sale_to_product_sheet('Bose SoundSport Headphones')
print('Output : ', product_identified)

Input :  Bose SoundSport Headphones
. Category identified :  headphone
. Brand identified :  bose
Output :  Bose SoundSport Headphones
